In [1]:
import sys
import random
import logging
import pandas as pd
from copy import deepcopy
from typing import List, OrderedDict

sys.path.append("..")

from magic_the_gathering.game_modes.default import DefaultGameMode
from magic_the_gathering.players.random import RandomPlayer
from magic_the_gathering.cards.deck_creator import RandomVanillaDeckCreator
from magic_the_gathering.game_state import GameState, ZonePosition
from magic_the_gathering.actions.base import Action
from magic_the_gathering.actions.none import NoneAction
from magic_the_gathering.actions.draw import DrawAction
from magic_the_gathering.game_engine import GameEngine
from magic_the_gathering.exceptions import GameOverException
from magic_the_gathering.turn import Turn
from magic_the_gathering.phases.players_get_priority import PhaseWherePlayersGetPriority

In [2]:
game_mode = DefaultGameMode()

In [3]:
game_logs_dataset = None

In [4]:
players = [
    RandomPlayer(
        index=0,
        life_points=game_mode.initial_life_points,
        game_logs_dataset=game_logs_dataset
    ),
    RandomPlayer(
        index=1,
        life_points=game_mode.initial_life_points,
        game_logs_dataset=game_logs_dataset
    )
]

In [5]:
def create_decks(
    n_players: int = 2,
) -> List[List[object]]:  # FIXME: I had to remove Card because it caused a circular import
    legal_lands_df = pd.read_csv("../data/basic_land_cards.csv")
    legal_creatures_df = pd.read_csv("../data/vanilla_creature_cards.csv")
    deck_creator = RandomVanillaDeckCreator(
        legal_lands_df,
        legal_creatures_df,
        deck_size=60,
        lands_proportion=0.4,
    )
    decks = deck_creator.create_decks(n_players=n_players)
    for i, deck in enumerate(decks):
        items = list(deck.items())
        random.shuffle(items)
        decks[i] = OrderedDict(items)
    # TODO: Need to add Mulligan phase
    return decks

In [6]:
decks = create_decks(n_players=len(players))

In [7]:
def create_hands(game_state: GameState):
    for player_index, player in enumerate(game_state.players):
        for _ in range(game_state.game_mode.initial_hand_size):
            game_state = DrawAction(player_index=player_index).execute(game_state=game_state)
    return game_state

In [8]:
game_state = GameState(
    game_mode=game_mode,
    players=players,
)
game_state.set_libraries(libraries=decks)
game_state = create_hands(game_state=game_state)

In [9]:
Action.HISTORY = []

In [10]:
logging.basicConfig(level=logging.DEBUG)

In [17]:
class ActionTree:
    def __init__(
        self,
        game_state: GameState,
        max_depth: int = None,
        depth: int = 0,
        chosen_action: Action = None,
        parent=None,
        children=None,
        winner_player_index: int = None
    ):
        self.game_state = game_state
        self.chosen_action = chosen_action
        self.max_depth = max_depth
        self.depth = depth
        self.parent = parent
        self.children = children
        if self.children is None:
            self.children = []
        self.winner_player_index = winner_player_index

    def add_child(self, action_tree):
        self.children.append(action_tree)
        action_tree.parent = self

    def build(self, phase_index: int, game_state: GameState):
        if self.max_depth is not None and self.depth == self.max_depth:
            return
        phase_index, game_state = self.__find_next_phase_index(
            phase_index=phase_index,
            chosen_action=self.chosen_action,
            game_state=game_state
        )
        current_phase = Turn.PHASES[phase_index]
        assert isinstance(current_phase, PhaseWherePlayersGetPriority)
        possible_actions = current_phase.list_possible_actions(game_state=game_state)
        for action in possible_actions:
            winner_player_index = None
            game_state_clone = deepcopy(game_state)
            try:
                game_state_clone = action.execute(game_state_clone)
            except GameOverException as e:
                winner_player_index = e.winner_player_index
            new_action_tree = ActionTree(
                game_state=game_state_clone,
                chosen_action=action,
                depth=self.depth + 1,
                max_depth=self.max_depth,
                winner_player_index=winner_player_index
            )
            self.add_child(new_action_tree)
            new_action_tree.build(
                phase_index=phase_index,
                game_state=game_state_clone
            )

    def __find_next_phase_index(self, phase_index: int, chosen_action: Action, game_state: GameState):
        if chosen_action is not None and not isinstance(chosen_action, NoneAction):
            return phase_index, game_state
        while True:
            phase_index = (phase_index + 1) % len(Turn.PHASES)
            phase = Turn.PHASES[phase_index]
            if isinstance(phase, PhaseWherePlayersGetPriority):
                break
            game_state = phase.run(game_state)
        return phase_index, game_state

    def draw(self):
        indent = "    " * self.depth
        print(f"{indent}{self.chosen_action.__class__.__name__} (current_player_index={self.game_state.current_player_index} | winner_player_index={self.winner_player_index})")
        for child in self.children:
            child.draw()

In [21]:
action_tree = ActionTree(
    game_state=game_state,
    max_depth=5
)

In [25]:
action_tree.build(phase_index=4, game_state=game_state)

INFO:CombatBeginningPhase:===== Combat: Beginning Phase =====
DEBUG:NoneAction:Executing action: NoneAction(source_player_index=0, target_player_index=None, source_card_uuids=None, target_card_uuids=None, source_zone=None, target_zone=None)


In [26]:
action_tree.draw()

NoneType (current_player_index=0 | winner_player_index=None)
    NoneAction (current_player_index=0 | winner_player_index=None)
        NoneAction (current_player_index=0 | winner_player_index=None)
    PlayLandAction (current_player_index=0 | winner_player_index=None)
        NoneAction (current_player_index=0 | winner_player_index=None)
            NoneAction (current_player_index=0 | winner_player_index=None)
        TapAction (current_player_index=0 | winner_player_index=None)
            NoneAction (current_player_index=0 | winner_player_index=None)
                NoneAction (current_player_index=0 | winner_player_index=None)
    PlayLandAction (current_player_index=0 | winner_player_index=None)
        NoneAction (current_player_index=0 | winner_player_index=None)
            NoneAction (current_player_index=0 | winner_player_index=None)
        TapAction (current_player_index=0 | winner_player_index=None)
            NoneAction (current_player_index=0 | winner_player_index=None